# Implementación del Modelo Predictivo en un Pipeline Automático

En este notebook, se desarrolla un pipeline completo y automatizado para la predicción de salarios en el sector tecnológico. Este pipeline integra la transformación de datos, el escalado, y el modelo de predicción en un único flujo de trabajo, optimizando el rendimiento y garantizando la reproducibilidad. Los pasos clave implementados son:

#### 1) Preprocesamiento Personalizado:
- Se diseñó una clase específica, CustomPreprocessor, que realiza múltiples tareas de transformación:
    - Mapeos Ordinales y Categóricos: Traduce valores textuales (como nivel educativo o frecuencia de actividades) a representaciones numéricas utilizando diccionarios definidos previamente.
    - Codificación One-Hot y MultiLabel: Convierte columnas categóricas y con múltiples etiquetas (como tecnologías y lenguajes utilizados) en representaciones binarizadas para su uso en modelos de machine learning.
    - Target Encoding: Asigna un valor promedio basado en la variable objetivo (`CompTotal`) para ciertas categorías seleccionadas, incorporando información sobre su relación directa con la variable objetivo.
    - Tratamiento de Valores Faltantes: Imputa valores ausentes según la naturaleza de cada columna (mediana, moda o valores predefinidos).
#### 2) Escalado y Modelo:
- Se utiliza un MinMaxScaler para normalizar las características numéricas, asegurando que todas las variables estén dentro del mismo rango.
- El modelo principal es un VotingRegressor que combina tres algoritmos base: Random Forest, Gradient Boosting, y XGBoost. Este modelo se entrena previamente y se integra directamente al pipeline
#### 3) Pipeline Final:
- El pipeline integra el preprocesador personalizado (`CustomPreprocessor`), el escalador, y el modelo de predicción en un único flujo de trabajo. Este diseño asegura que los datos brutos puedan ser transformados y utilizados para predicciones de manera consistente y sin pasos manuales intermedios.
#### 4) Manejo de Datos:
- Se combinan los datasets de las encuestas de Stack Overflow de los años 2023 y 2024, seleccionando las columnas más relevantes y filtrando los datos según un rango razonable de salarios (18,000 a 55,000 euros).
- La variable objetivo (CompTotal) se transforma logarítmicamente para manejar su distribución asimétrica y mejorar el rendimiento del modelo.
#### 5) Exportación del Pipeline:
- Una vez entrenado, el pipeline completo se guarda en un archivo mediante joblib, permitiendo su reutilización para nuevas predicciones sin necesidad de repetir el proceso de preprocesamiento y entrenamiento.

### Propósito de la Clase CustomPreprocessor
La clase `CustomPreprocessor` es un componente clave del pipeline que permite procesar datos brutos y prepararlos para el modelo. Esta clase ofrece las siguientes ventajas:
- Automatización del Preprocesamiento: Integra múltiples técnicas de transformación en una sola clase, reduciendo la necesidad de pasos manuales.
- Adaptabilidad: Permite manejar columnas categóricas, ordinales, y multilabel con facilidad, incorporando target encoding y codificación binarizada según sea necesario.
- Escalabilidad: Diseñada para manejar grandes conjuntos de datos con múltiples columnas de diferentes tipos.

In [5]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
import xgboost as xgb
import pickle
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
import joblib
import os
from custom_preprocessor import CustomPreprocessor  # Importar clase personalizada para preprocesamiento

# Mapeo de niveles educativos a valores ordinales
edlevel_mapping = {
    'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 5,
    'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 4,
    'Professional degree (JD, MD, Ph.D, Ed.D, etc.)': 6,
    'Some college/university study without earning a degree': 2,
    'Secondary school (e.g. American high school)': 1,
    'Associate degree (A.A., A.S., etc.)': 3,
    'Something else': -1,
    'Primary/elementary school': 0
}

# Mapeo de industrias a categorías generalizadas
industry_mapping = {
    'Information Services, IT, Software Development, or other Technology': 'Tecnología y Servicios Digitales',
    'Healthcare': 'Salud y Educación',
    'Retail and Consumer Services': 'Otros Servicios',
    'Legal Services': 'Otros Servicios',
    'Higher Education': 'Salud y Educación',
    'Financial Services': 'Servicios Financieros',
    'Manufacturing': 'Industria y Energía',
    'Insurance': 'Servicios Financieros',
    'Oil & Gas': 'Industria y Energía'
}

# Mapeo de frecuencia a valores ordinales
frequency_mapping = {
    '10+ times a week': 4, 
    '6-10 times a week': 3, 
    '3-5 times a week': 2,
    '1-2 times a week': 1, 
    'Never': 0, 
    'Other': -1
}

# Columnas objetivo para encoding personalizado
target_columns = ['LearnCodeOnline', 'DevType', 'LearnCode', 'CodingActivities']

# Columnas multilabel para codificación binarizada
multi_label_columns = ['DatabaseHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageHaveWorkedWith', 'ToolsTechHaveWorkedWith']

# Inicialización del preprocesador personalizado
preprocessor = CustomPreprocessor(target_columns, multi_label_columns, edlevel_mapping, industry_mapping, frequency_mapping)

# Cargar el scaler y modelo previamente entrenados desde archivos
ruta_pickles = os.path.join("..", "Pickles")
ruta_scaler = os.path.join(ruta_pickles, 'Scaler.pkl')
ruta_modelo = os.path.join(ruta_pickles, 'MiModelo.pkl')

scaler = joblib.load(ruta_scaler)  # Carga del escalador
voting_regressor = joblib.load(ruta_modelo)  # Carga del modelo de votación

# Definición del pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesador personalizado
    ('scaler', scaler),              # Escalador para normalización
    ('model', voting_regressor)      # Modelo final
])

# Cargar datos de entrada desde archivos pickle
with open('../Pickles/data_2023.pickle', 'rb') as archivo:   
    df1 = pickle.load(archivo)
with open('../Pickles/data_2024.pickle', 'rb') as archivo:
    df2 = pickle.load(archivo)

# Combinar datasets de años distintos
df = pd.concat([df1, df2], ignore_index=True, join='inner')

# Selección de columnas relevantes
df = df[['YearsCodePro', 'LearnCodeOnline', 'DevType', 'LearnCode', 'CodingActivities', 
         'DatabaseHaveWorkedWith', 'YearsCode', 'LanguageWantToWorkWith', 
         'LanguageHaveWorkedWith', 'EdLevel', 'Employment', 'ToolsTechHaveWorkedWith', 
         'AISent', 'Industry', 'Frequency_2', 'Frequency_1', 'CompTotal']]

# Filtrar datos según valores razonables en la variable objetivo (CompTotal)
limite_inferior = 18000
limite_superior = 55000
df = df[(df['CompTotal'] >= limite_inferior) & (df['CompTotal'] <= limite_superior)]

# Separar variable objetivo (y) y características (X)
y = np.log1p(df['CompTotal'])  # Transformación logarítmica de la variable objetivo
X = df.drop(columns=['CompTotal'])  # Eliminar CompTotal de las características

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el pipeline completo con los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Define la ruta para guardar el pipeline entrenado
ruta_pipeline = os.path.join(ruta_pickles, "Pipeline.pkl")
os.makedirs(ruta_pickles, exist_ok=True)  # Crear carpeta si no existe

# Guardar el pipeline en un archivo
joblib.dump(pipeline, ruta_pipeline)

# Confirmación de guardado exitoso
print(f"Pipeline guardado exitosamente en: {ruta_pipeline}")

Pipeline guardado exitosamente en: ..\Pickles\Pipeline.pkl


# Predicciones con nuevas muestras

Este notebook representa el cierre del proceso de desarrollo del modelo predictivo, ofreciendo una solución completamente funcional y automatizada para la predicción de salarios. Con el pipeline entrenado y exportado, el siguiente paso será probar su rendimiento con nuevos datos de entrada, validando su capacidad de generalización en escenarios reales.

Creo una nueva muestra de 3 variables para probar predicciones. 

In [6]:
new_data = pd.DataFrame({
    'YearsCodePro': [3, 5, 10],
    'LearnCodeOnline': ['How-to videos;Online Forum', 'How-to videos;Online Forum', 'Online Forum'],
    'DevType': ['Developer, back-end', 'Developer, full-stack', 'Developer, mobile'],
    'LearnCode': ['On the job training', 'On the job training;School (i.e., University, College, etc)', 'On the job training;School (i.e., University, College, etc)'],
    'CodingActivities': ['Freelance/contract work', 'Hobby;Freelance/contract work', 'Hobby;Contribute to open-source projects'],
    'DatabaseHaveWorkedWith': ['MySQL', 'PostgreSQL;MySQL', 'PostgreSQL;SQLite'],
    'YearsCode': [5, 10, 15],
    'LanguageWantToWorkWith': ['Python;JavaScript', 'Java;Python', 'C++;Python'],
    'LanguageHaveWorkedWith': ['Python;JavaScript', 'Java;Python;SQL', 'C++;Python;SQL'],
    'EdLevel': ['Master’s degree (M.A., M.S., M.Eng., MBA, etc.)', 'Bachelor’s degree (B.A., B.S., B.Eng., etc.)', 'Some college/university study without earning a degree'],
    'Employment': ['Employed, full-time', 'Employed, full-time', 'Employed, full-time'],
    'ToolsTechHaveWorkedWith': ['Docker', 'Docker;Kubernetes', 'Git'],
    'AISent': ['Favorable', 'Unsure', 'Favorable'],
    'Industry': ['Information Services, IT, Software Development, or other Technology', 'Financial Services', 'Manufacturing'],
    'Frequency_2': ['3-5 times a week', '1-2 times a week', 'Never'],
    'Frequency_1': ['1-2 times a week', 'Never', '6-10 times a week']
})

A continuación transformo los datos nuevos, adaptando los nombres de las columnas y generando un DataFrame compatible con el pipeline, asegurando que los datos estén en el formato esperado para el modelo.
1) Utilizo el preprocesador del pipeline (CustomPreprocessor) para aplicar las transformaciones definidas previamente. Estas transformaciones incluyen la codificación de variables categóricas, el mapeo de valores ordinales, la binarización de variables multilabel, entre otras, adaptando los datos a la estructura requerida por el modelo.

2) Se genera una lista de los nombres de las columnas resultantes después del preprocesamiento. Esto es útil porque las transformaciones pueden cambiar los nombres de las columnas originales (por ejemplo, al crear columnas codificadas con one-hot encoding o binarización).

3) Convierte los datos preprocesados en un nuevo DataFrame, asignando los nombres de las columnas generados en el paso anterior. Este DataFrame es estructurado y listo para ser utilizado como entrada al modelo de predicción.

4) Relleno los Nan con 0

In [7]:
# Transformar los datos nuevos (new_data) con el preprocesador
preprocessed_data = pipeline.named_steps['preprocessor'].transform(new_data)
# Obtener los nombres de las columnas resultantes
column_names = pipeline.named_steps['preprocessor'].get_feature_names_out(input_features=new_data.columns)
# Crear un DataFrame con los datos preprocesados
preprocessed_df = pd.DataFrame(preprocessed_data, columns=column_names)
preprocessed_df = preprocessed_df.fillna(0)

Luego utilizo el modelo final del pipeline para predecir el salario basado en los datos preprocesados contenidos en preprocessed_df. 

Este fragmento aplica el modelo de predicción, convierte los resultados de la escala logarítmica a la escala original, y los presenta en una serie con valores de salario estimados. Es el paso final para obtener predicciones utilizables del modelo entrenado.

1) El modelo devuelve los valores de predicción en escala logarítmica porque el objetivo (CompTotal) se transformó con np.log1p durante el entrenamiento.

2) Transformo los valores de predicción de vuelta a la escala original del salario utilizando la función np.expm1. Esto revierte la transformación logarítmica aplicada previamente, haciendo que los resultados sean interpretables como salarios en su unidad original.

3) Muestro la serie resultante con las predicciones del salario en su escala original. Cada valor de esta serie corresponde a la predicción del salario para una fila en los datos de entrada new_data.

In [8]:
# Realizar la predicción con las columnas finales
predicted_salary_log = pipeline.named_steps['model'].predict(preprocessed_df)

# Convertir de escala logarítmica a escala original
predicted_salary = pd.Series(np.expm1(predicted_salary_log), name="Predicted Salary")

# Verificar resultados
print(predicted_salary)

0    22439.231636
1    23256.987311
2    22392.101585
Name: Predicted Salary, dtype: float64


c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
c:\Users\emita\.conda\envs\proyecto_ml\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


En el siguiente notebook haré un Streamlit para poder desplegar el modelo en una web. 